Use Python to fetch XAUUSDc H1 candles data from MT5. UTC based timezone.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight session.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight sessions. Make a developing RSI indicator that starts calculating from the start of the trading day and resets at the close of the trading day. PLot the tivk-volume bars on separate pane, plot the RSI indicaor in seeparate pane.